In [1]:
import random
import datetime
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
df = pd.read_excel("交易資料集.xlsx")

In [3]:
df

,ITEM_ID,ITEM_NO,PRODUCT_TYPE,CUST_ID,TRX_DATE,INVOICE_NO,QUANTITY
0,3217532,M25P40-VMN6TPB,MEMORY_EMBEDED,3218,2016-07-26,CX47348203,2500
1,3326781,AU80610006237AASLBX9,CPU / MPU,2470,2016-07-11,CX47346522,50
2,740487,MMBD2837LT1G,DISCRETE,16135,2016-07-27,CX47348534,3000
3,3434776,IHLP1616ABER2R2M11,PEMCO,999999999,2016-07-29,A20160700174,0
4,70072,MMBT3906LT1G,DISCRETE,2356,2016-07-06,CX47346184,12000
...,...,...,...,...,...,...,...
157391,133619,"74LVC2G17GW,125",LOGIC IC,2579,2016-07-11,216072287,1644
157392,15231095,MT41K256M16TW-107:P TRAY,MEMORY_EMBEDED,43262,2016-07-28,216077517,300000
157393,14864010,MT36KSF2G72PZ-1G6E1FE,MEMORY_EMBEDED,47302,2016-07-07,A10123295,-1
157394,15220039,CM8066201920103SR2L2,CPU / MPU,49263,2016-07-22,A10123946,-2


In [4]:
#將Quantity<=0的值刪除
for i in range(len(df["QUANTITY"])):
    if df["QUANTITY"][i]<=0:
        df=df.drop(i, axis=0)

In [5]:
df

,ITEM_ID,ITEM_NO,PRODUCT_TYPE,CUST_ID,TRX_DATE,INVOICE_NO,QUANTITY
0,3217532,M25P40-VMN6TPB,MEMORY_EMBEDED,3218,2016-07-26,CX47348203,2500
1,3326781,AU80610006237AASLBX9,CPU / MPU,2470,2016-07-11,CX47346522,50
2,740487,MMBD2837LT1G,DISCRETE,16135,2016-07-27,CX47348534,3000
4,70072,MMBT3906LT1G,DISCRETE,2356,2016-07-06,CX47346184,12000
6,3420352,TMP103AYFFR,LINEAR IC,10228,2016-07-25,CX47347899,3000
...,...,...,...,...,...,...,...
157389,14691206,"74HCT365D,653",LOGIC IC,2100,2016-07-06,216071271,7030
157390,3249493,"74HC04D,653",LOGIC IC,2197,2016-07-21,216075314,347
157391,133619,"74LVC2G17GW,125",LOGIC IC,2579,2016-07-11,216072287,1644
157392,15231095,MT41K256M16TW-107:P TRAY,MEMORY_EMBEDED,43262,2016-07-28,216077517,300000


In [6]:
df=df.drop("ITEM_NO", axis=1)
df=df.drop("PRODUCT_TYPE", axis=1)
df=df.drop("CUST_ID", axis=1)
df=df.drop("TRX_DATE", axis=1)
df=df.drop("QUANTITY", axis=1)

In [7]:
df = df[['INVOICE_NO','ITEM_ID']]
df=df.reset_index(drop=True)
df

,INVOICE_NO,ITEM_ID
0,CX47348203,3217532
1,CX47346522,3326781
2,CX47348534,740487
3,CX47346184,70072
4,CX47347899,3420352
...,...,...
128832,216071271,14691206
128833,216075314,3249493
128834,216072287,133619
128835,216077517,15231095


In [8]:
#將資料集內之整數轉字串
df['INVOICE_NO'] = df['INVOICE_NO'].astype(str)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128837 entries, 0 to 128836
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   INVOICE_NO  128837 non-null  object
 1   ITEM_ID     128837 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.0+ MB


In [10]:
#找最大類別個數
result=pd.value_counts(list(df['INVOICE_NO']))
result

216077144     530
E216070037    511
E216070036    468
216077177     424
216074165     399
             ... 
316075668       1
316075190       1
1616070464      1
316072684       1
216077517       1
Length: 36498, dtype: int64

In [11]:
#交易訂單總類別個數
a=list(set(df['INVOICE_NO']))
len(list(set(df['INVOICE_NO'])))

36498

In [12]:
a

['316073586',
 '216077085',
 'A716070149/17830718',
 'W016074741',
 '316073540',
 'W016078435',
 'CX47346007',
 'CX47345256',
 'W016073839',
 '1616070449/79213985',
 'W016074133',
 '1616071400/88761765',
 '2316070086',
 '316070796',
 'CX48010275',
 '306071345',
 'K516070234',
 '216075875',
 '216077588',
 'CX47347473',
 '316072449',
 '316070962',
 '316074669',
 '1616070773/普00871192',
 '3811600124',
 '316075405',
 '216077653',
 '316072560',
 '3416070209',
 '316076019',
 '1916070143',
 '216072554',
 'CX47345289',
 '316079947',
 '316073493',
 '316078943',
 'W016070622',
 'W016072066',
 '316073823',
 '3416070346',
 'CX47346014',
 '1616070674/普00871189',
 'CX47345795',
 'CX47347069',
 'W016072769',
 '216072924',
 'O516070868',
 '316074483',
 '216073167',
 '306071059',
 'W016072000',
 '216074316',
 '216076450',
 '3416070373',
 '2316070199',
 '316077805',
 'CX47346821',
 '306071382',
 '306071524',
 'CX47346191',
 'O516070173',
 '6F01600279',
 'W016075531',
 '216071605',
 '216077074',
 '316071

In [13]:
#產品總類別個數
a2=(list(set(df['ITEM_ID'])))
a2

[3,
 8,
 9,
 16,
 18,
 19,
 23,
 41,
 10092587,
 44,
 50,
 131127,
 61,
 63,
 74,
 77,
 78,
 80,
 131155,
 84,
 85,
 90,
 91,
 92,
 94,
 98,
 100,
 101,
 102,
 103,
 104,
 106,
 108,
 109,
 120,
 127,
 131,
 132,
 4456590,
 147,
 131231,
 131237,
 170,
 171,
 172,
 173,
 174,
 131252,
 131254,
 189,
 190,
 191,
 131266,
 195,
 131270,
 210,
 211,
 215,
 216,
 219,
 222,
 252,
 257,
 262,
 265,
 269,
 271,
 272,
 273,
 274,
 275,
 4194587,
 131382,
 314,
 131387,
 317,
 131392,
 131396,
 131399,
 334,
 336,
 131411,
 3408215,
 131417,
 346,
 349,
 131422,
 131424,
 354,
 355,
 131429,
 131438,
 374,
 131449,
 377,
 131451,
 385,
 386,
 15597960,
 15597966,
 15597968,
 15597969,
 15597972,
 404,
 405,
 15597974,
 15597976,
 15597973,
 15597978,
 412,
 3015068,
 413,
 415,
 416,
 417,
 15597989,
 15597991,
 3277227,
 15597996,
 15598011,
 3408329,
 3277258,
 131536,
 3408343,
 131543,
 15466977,
 3408359,
 2884071,
 2884072,
 3408381,
 3408385,
 3408398,
 15335975,
 131624,
 131627,
 1533

In [14]:
for i in range(len(a2)):
    a2[i]=str(a2[i])
a2

['3',
 '8',
 '9',
 '16',
 '18',
 '19',
 '23',
 '41',
 '10092587',
 '44',
 '50',
 '131127',
 '61',
 '63',
 '74',
 '77',
 '78',
 '80',
 '131155',
 '84',
 '85',
 '90',
 '91',
 '92',
 '94',
 '98',
 '100',
 '101',
 '102',
 '103',
 '104',
 '106',
 '108',
 '109',
 '120',
 '127',
 '131',
 '132',
 '4456590',
 '147',
 '131231',
 '131237',
 '170',
 '171',
 '172',
 '173',
 '174',
 '131252',
 '131254',
 '189',
 '190',
 '191',
 '131266',
 '195',
 '131270',
 '210',
 '211',
 '215',
 '216',
 '219',
 '222',
 '252',
 '257',
 '262',
 '265',
 '269',
 '271',
 '272',
 '273',
 '274',
 '275',
 '4194587',
 '131382',
 '314',
 '131387',
 '317',
 '131392',
 '131396',
 '131399',
 '334',
 '336',
 '131411',
 '3408215',
 '131417',
 '346',
 '349',
 '131422',
 '131424',
 '354',
 '355',
 '131429',
 '131438',
 '374',
 '131449',
 '377',
 '131451',
 '385',
 '386',
 '15597960',
 '15597966',
 '15597968',
 '15597969',
 '15597972',
 '404',
 '405',
 '15597974',
 '15597976',
 '15597973',
 '15597978',
 '412',
 '3015068',
 '413',
 

In [15]:
## 交易訂單總個數
b=len(df['INVOICE_NO'])
b

128837

In [17]:
#前處理(會花好幾小時)
df3=[]
df4=[]
for i in range(len(a)):
    df4=[]
    for j in range(b):
        if a[i]==df['INVOICE_NO'][j]:
            df4.append(df['ITEM_ID'][j].astype(np.int64))
    df4=list(set(df4))
    df4.sort()
    df4=list(map(str,df4))
    df3.append(df4)

In [20]:
df3

# 儲存結果

In [ ]:
newdf=df3
newdf=np.array(newdf)
np.save('newdf_repeat.npy',newdf) # 保存為.npy格式

# 讀取結果

In [16]:
df3=np.load('newdf_str.npy',allow_pickle=True)
df3=df3.tolist()
df3

[['14931950', '14957868', '14957869', '15318063'],
 ['1716',
  '70099',
  '77034',
  '107040',
  '129417',
  '130474',
  '134290',
  '152311',
  '179051',
  '1160730'],
 ['227540'],
 ['3216409'],
 ['3329259'],
 ['11361588'],
 ['14885890'],
 ['3417453'],
 ['14982998', '14983002'],
 ['3249203'],
 ['14511731'],
 ['142994'],
 ['12560593'],
 ['225608'],
 ['3403838'],
 ['183067', '1175731'],
 ['7687587', '15481971'],
 ['3342337'],
 ['2249019',
  '3387057',
  '14797241',
  '14797243',
  '14797244',
  '14799325',
  '14799326',
  '14799327',
  '14799334'],
 ['3190569'],
 ['15352007'],
 ['14839016'],
 ['14532748'],
 ['2205697'],
 ['14710547'],
 ['14677900'],
 ['14478725'],
 ['14415734', '14844111'],
 ['15490962'],
 ['87141', '149026', '149269', '195115', '3423375', '3423985'],
 ['3399221'],
 ['3196210'],
 ['14863021'],
 ['14605751',
  '14782210',
  '15141363',
  '15189014',
  '15280990',
  '15288975',
  '15288976'],
 ['3216512'],
 ['15280990'],
 ['3320895'],
 ['15280990'],
 ['6642591', '15078912

# 資料處理

In [17]:
## Import Package
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
## Data 自行定義數據
data = {'Items':df3}
## 轉成DataFrame
data = pd.DataFrame(data)
## 讓DataFrame 能呈現的寬度大一點
pd.options.display.max_colwidth = 100
## 轉成數值編碼，目前都是字串的組合
data_id=pd.DataFrame([np.NaN]*1 for i in range(len(a)))
for i in range(len(a)):
    data_id[0][i]=str(a[i])
    data['Items'][i]=list(map(str,data['Items'][i]))
data_id.columns=['Transaction ID']
data

,Items
0,"[14931950, 14957868, 14957869, 15318063]"
1,"[1716, 70099, 77034, 107040, 129417, 130474, 134290, 152311, 179051, 1160730]"
2,[227540]
3,[3216409]
4,[3329259]
...,...
36493,[8713587]
36494,[15078912]
36495,[15411003]
36496,"[122892, 122967, 2578015, 3184268, 3221993, 3342398, 3387581, 3399385, 4264926, 14556818]"


In [18]:
data_items = data.Items.str.join(',')
## 轉成數值
data_items = data_items.str.get_dummies(',')
## 接上Transaction ID
data = data_id.join(data_items)
data

,Transaction ID,100,1000,100016,100018,10002587,100026,100030,10018587,10021587,...,995630,995631,9964587,996629,996630,9971587,9972587,9980587,999,9999587
0,316073586.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,216077085.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,A716070149/17830718,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,W016074741,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,316073540,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36493,216078133,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36494,316073756,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36495,616070994,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36496,1616071064/普00871198,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Apriori

In [21]:
#起始時間
start = datetime.datetime.now()
#訓練
## 計算支持度 Support
Support_items = apriori(data[a2], min_support=0.0007, max_len=3, use_colnames = True,low_memory=True)
## 計算關聯規則 Association Rule
Association_Rules = association_rules(Support_items, metric = 'lift', min_threshold=1)
#結束時間
end = datetime.datetime.now()
#輸出程式花費時間
print("執行時間：", end - start)
#Association_Rules.to_excel("Apriori_min_support0.001.xlsx")
Association_Rules

C:\Users\FUZZY_LAB\.conda\envs\NEW\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:115: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  DeprecationWarning,


執行時間： 0:00:33.865452


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(15336004),(15336088),0.002822,0.002466,0.000877,0.310680,125.990939,0.000870,1.447127
1,(15336088),(15336004),0.002466,0.002822,0.000877,0.355556,125.990939,0.000870,1.547345
2,(14790248),(15336004),0.005315,0.002822,0.000904,0.170103,60.275948,0.000889,1.201568
3,(15336004),(14790248),0.002822,0.005315,0.000904,0.320388,60.275948,0.000889,1.463607
4,(14929876),(15336004),0.002603,0.002822,0.000712,0.273684,96.979867,0.000705,1.372926
...,...,...,...,...,...,...,...,...,...
1425,"(88622, 88623)",(88676),0.001260,0.001507,0.000712,0.565217,375.078261,0.000710,2.296534
1426,"(88676, 88623)",(88622),0.000849,0.001397,0.000712,0.838710,600.220114,0.000711,6.191337
1427,(88622),"(88676, 88623)",0.001397,0.000849,0.000712,0.509804,600.220114,0.000711,2.038267
1428,(88676),"(88622, 88623)",0.001507,0.001260,0.000712,0.472727,375.078261,0.000710,1.894161


# FP-GROWTH

In [26]:
from mlxtend.frequent_patterns import fpgrowth

In [27]:
#起始時間
start = datetime.datetime.now()
#訓練
## 計算支持度 Support
frequent_itemsets=fpgrowth(data[a2], min_support=0.0007, use_colnames=True)
## 計算關聯規則 Association Rule
rules=association_rules(frequent_itemsets,metric='lift',min_threshold=1)
#結束時間
end = datetime.datetime.now()
#輸出程式花費時間
print("執行時間：", end - start)
rules.to_excel("FP-Growth_min_support0.001.xlsx")
rules

C:\Users\FUZZY_LAB\.conda\envs\NEW\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:115: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  DeprecationWarning,


執行時間： 0:00:28.585460


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(70509),(70449),0.008137,0.005206,0.001233,0.151515,29.105263,0.001191,1.172436
1,(70449),(70509),0.005206,0.008137,0.001233,0.236842,29.105263,0.001191,1.299682
2,(23004),(70509),0.004740,0.008137,0.001699,0.358382,44.041105,0.001660,1.545876
3,(70509),(23004),0.008137,0.004740,0.001699,0.208754,44.041105,0.001660,1.257839
4,(23004),(1708),0.004740,0.004822,0.001343,0.283237,58.736272,0.001320,1.388434
...,...,...,...,...,...,...,...,...,...
137,(14876857),(14870535),0.001123,0.001096,0.001096,0.975610,890.195122,0.001095,40.955066
138,(15157064),(14481833),0.002055,0.001507,0.001123,0.546667,362.768000,0.001120,2.202558
139,(14481833),(15157064),0.001507,0.002055,0.001123,0.745455,362.768000,0.001120,3.920499
140,(14671860),(15192100),0.001644,0.001891,0.001096,0.666667,352.637681,0.001093,2.994328


# 查詢規則

In [31]:
user_input = list(input("請輸入商品編號(不同商品以空白間隔開)").split(', '))
print(user_input)
Arule = pd.DataFrame()
Arule2=set()
Frule = pd.DataFrame()
Frule2=set()
#尋找Apriori中輸入的產品編號和關連表中的關聯規則
print("Apriori:")
for i in range(len(Association_Rules["antecedents"])):
    if frozenset(user_input) <= Association_Rules["antecedents"][i]:
        Arule=Arule.append(Association_Rules.iloc[[i]],ignore_index=True)
if len(Arule)==0:
    print("您輸入的產品編號無相關推薦產品！")
elif len(Arule["consequents"])==1:
    print("推薦商品編號為",list(Arule["consequents"][0]))
else:
    #刪除冗餘規則
    for i in range(len(Arule["consequents"])):
        for j in range(len(Arule["consequents"])-i-1):
            if Arule["antecedents"][i]<=Arule["antecedents"][j] and Arule["consequents"][i]==Arule["consequents"][j] and Arule["confidence"][i]>=Arule["confidence"][j]:
                Arule.drop(i)
            if Arule["antecedents"][i]>=Arule["antecedents"][j] and Arule["consequents"][i]==Arule["consequents"][j] and Arule["confidence"][i]<=Arule["confidence"][j]:
                Arule.drop(j)
            Arule=Arule.reset_index(drop=True)
    #將重複的商品刪除
    for i in range(len(Arule["consequents"])-1):
        Arule2=Arule2.union(Arule["consequents"][i],Arule["consequents"][i+1])
    print("推薦商品編號為",list(Arule2))
Arule2.clear()

#尋找FP-GROWTH中輸入的產品編號和關連表中的關聯規則
print("FP-GROWTH:")
for i in range(len(rules["antecedents"])):
    if frozenset(user_input) <= rules["antecedents"][i]:
        Frule=Frule.append(rules.iloc[[i]],ignore_index=True)
if len(Frule)==0:
    print("您輸入的產品編號無相關推薦產品！")
elif len(Frule["consequents"])==1:
    print("推薦商品編號為",list(Frule["consequents"][0]))
else:
    #刪除冗餘規則
    for i in range(len(Frule["consequents"])):
        for j in range(len(Frule["consequents"])-i-1):
            if Frule["antecedents"][i]<=Frule["antecedents"][j] and Frule["consequents"][i]==Frule["consequents"][j] and Frule["confidence"][i]>=Frule["confidence"][j]:
                Frule.drop(i)
            if Frule["antecedents"][i]>=Frule["antecedents"][j] and Frule["consequents"][i]==Frule["consequents"][j] and Frule["confidence"][i]<=Frule["confidence"][j]:
                Frule.drop(j)
            Frule=Frule.reset_index(drop=True)
    #將重複的商品刪除
    for i in range(len(Frule["consequents"])-1):
        Frule2=Frule2.union(Frule["consequents"][i],Frule["consequents"][i+1])
    print("推薦商品編號為",list(Frule2))
Frule2.clear()
print(Arule)
print(Frule)

請輸入商品編號(不同商品以空白間隔開)15157064, 15192100
['15157064', '15192100']
Apriori:
推薦商品編號為 ['14481833']
FP-GROWTH:
您輸入的產品編號無相關推薦產品！
            antecedents consequents  antecedent support  consequent support  \
0  (15157064, 15192100)  (14481833)            0.000932            0.001507   

    support  confidence        lift  leverage  conviction  
0  0.000712    0.764706  507.458824  0.000711    4.243596  
Empty DataFrame
Columns: []
Index: []
